# Mission 10 · Text Embedding & Classification Reference

Baseline 코드를 확장해 **Word2Vec / FastText / GloVe** 임베딩 + **RNN 분류기** 실험을 한 번에 수행할 수 있는 노트북입니다. 각 단계는 마크다운과 주석으로 상세히 설명되어 있어, 셀을 순서대로 실행하며 구조를 그대로 활용하거나 응용할 수 있습니다.

## 1. 기본 설정
- 필수 라이브러리 임포트
- GPU / CPU 디바이스 확인
- 재현성을 위한 시드 고정

In [ ]:
# 표준 라이브러리: 경로/난수/정규식/카운팅/데이터구조/경로객체/타입힌트
import os                   # 📁 파일/디렉터리 경로 조작 (이번 코드에선 주로 경로 상수/존재확인 등에 활용 가능)
import random               # 🎲 파이썬 난수 발생기(시드 고정으로 재현성 확보)
import re                   # ✂️ 정규표현식(텍스트 전처리 시 특수문자 제거 등)
from collections import Counter  # 🔢 토큰 빈도수 집계(단어 분포 확인에 유용)
from dataclasses import dataclass # 📦 실험 설정/하이퍼파라미터를 구조체로 묶을 때 유용
from pathlib import Path     # 🛣️ OS 독립적인 경로 처리
from typing import Dict, List, Tuple  # 🏷️ 가독성·정확성을 위한 타입 힌트

# 수치/딥러닝/평가 관련 라이브러리
import numpy as np           # 🔢 벡터/행렬 연산(임베딩 평균, 스케일링 등)
import torch                 # 🔥 PyTorch 텐서/자동미분/디바이스 관리
import torch.nn as nn        # 🧠 신경망 레이어/손실함수 등
from sklearn.datasets import fetch_20newsgroups  # 📰 20개 뉴스그룹 데이터 로더
from sklearn.metrics import accuracy_score, classification_report  # 📊 성능지표 계산
from sklearn.model_selection import train_test_split              # 🔀 학습/검증 데이터 분할
from torch.utils.data import DataLoader, Dataset                  # 🚚 배치 로딩/커스텀 데이터셋

# 임베딩/토큰화
from gensim.models import Word2Vec, FastText   # 🧩 단어 임베딩 학습(Word2Vec/서브워드 기반 FastText)
from nltk.corpus import stopwords              # 🧹 불용어 리스트 (전처리)
from nltk.tokenize import word_tokenize        # ✂️ 단어 단위 토큰화
import nltk                                    # 🧰 NLTK 리소스 다운로드/관리

# (선택) GloVe 사전학습 임베딩: torchtext가 설치/호환될 때만 사용
try:
    from torchtext.vocab import GloVe          # 📦 사전 학습된 GloVe 벡터 로더
    TORCHTEXT_AVAILABLE = True                 # ✅ 사용 가능 플래그
except Exception:
    TORCHTEXT_AVAILABLE = False                # ❌ 환경에 따라 미지원일 수 있으므로 안전 처리

# NLTK 리소스 다운로드(최초 1회 필요)
nltk.download('punkt')       # 🔻 토큰화에 필요한 데이터(마침표/구두점 규칙 등)
nltk.download('stopwords')   # 🔻 불용어 사전(영어 등 다수 언어)

# 재현성 확보: 시드 고정(난수에 의존하는 학습/샘플링 변동을 최소화)
SEED = 42
random.seed(SEED)            # 🎲 파이썬 표준 난수 시드
np.random.seed(SEED)         # 🎲 NumPy 난수 시드
torch.manual_seed(SEED)      # 🎲 PyTorch CPU 연산 난수 시드
torch.cuda.manual_seed_all(SEED)  # 🎲 (멀티)GPU 연산 난수 시드(드롭아웃/초기화 등 일관성)

# 학습 디바이스 선택: CUDA가 있으면 GPU 사용, 없으면 CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)  # 🖨️ 현재 사용 중인 디바이스 출력(디버깅/로그용)

## 2. 데이터 로드 & 탐색
`fetch_20newsgroups` 전체 데이터를 불러와 텍스트/레이블을 확인합니다.

In [ ]:
news_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = news_data.data
labels = news_data.target
label_names = news_data.target_names

print(f"총 문서 수: {len(texts)}")
print(f"레이블 개수: {len(label_names)}")
print('샘플 문서:', texts[0][:500])
print('샘플 레이블:', label_names[labels[0]])


## 3. 데이터 전처리 파이프라인
- 정규화/토큰화/불용어 제거 함수 정의
- 훈련/검증/테스트 분할
- 문장 길이 통계를 통해 패딩 길이를 결정

In [ ]:
# 🔹 영어 불용어(stopwords) 리스트를 불러와 집합(set) 형태로 저장
#   set으로 변환하는 이유: 검색 속도가 리스트보다 훨씬 빠르기 때문
stop_words = set(stopwords.words('english'))

# 🔹 정규표현식 패턴 정의
#   [^a-zA-Z']+  → 알파벳과 작은따옴표(')를 제외한 모든 문자들을 의미
#   즉, 특수문자/숫자/공백 등을 공백(' ')으로 대체하기 위한 패턴
TOKEN_PATTERN = re.compile(r"[^a-zA-Z']+")


def clean_and_tokenize(text: str) -> List[str]:
    """
    ✅ 텍스트 전처리 파이프라인 함수
    입력 문자열을 처리하여 단어 단위 리스트(List[str])로 반환합니다.
    
    주요 단계:
    1️⃣ 소문자 변환(lower)
    2️⃣ 특수문자 제거 (정규표현식으로)
    3️⃣ 단어 토큰화 (nltk의 word_tokenize 사용)
    4️⃣ 불용어(stopword) 및 한 글자 단어 제거
    """

    text = text.lower()                  # 1️⃣ 텍스트를 모두 소문자로 변환 → 대소문자 구분 제거
    text = TOKEN_PATTERN.sub(' ', text)  # 2️⃣ 알파벳/‘ 외의 문자를 공백으로 치환 → 특수문자 정제
    tokens = word_tokenize(text)         # 3️⃣ 문장을 단어 단위로 분리
    # 4️⃣ 불용어 및 한 글자(길이 1 이하) 단어 제거
    tokens = [tok for tok in tokens if tok not in stop_words and len(tok) > 1]
    return tokens                        # 5️⃣ 정제된 토큰 리스트 반환

# ------------------------------------------------------------
# ✅ 데이터셋 분리 단계
# ------------------------------------------------------------

# 🔹 1차 분할: Train/Test (전체 데이터 중 20%를 테스트로 분리)
#   stratify=labels → 클래스 비율을 유지한 채로 분리 (불균형 데이터 방지)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=SEED, stratify=labels
)

# 🔹 2차 분할: Train → Train/Validation (훈련 데이터 중 10%를 검증 세트로 분리)
#   stratify=train_labels → 학습 데이터 내에서도 클래스 균형 유지
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=SEED, stratify=train_labels
)

# 🔹 분할 결과 확인
#   train_texts: 학습용 문장 리스트
#   val_texts: 검증용 문장 리스트
#   test_texts: 테스트용 문장 리스트
print('Train/Val/Test sizes:', len(train_texts), len(val_texts), len(test_texts))



### 토큰화 & 길이 분석
길이 분포를 보고 최대 길이를 유연하게 결정할 수 있습니다.

In [ ]:
def tokenize_corpus(text_list: List[str]) -> List[List[str]]:
    # ✅ 입력: 문자열 리스트(text_list)
    # ✅ 출력: 각 문서를 토큰 리스트로 바꾼 2차원 리스트(List[List[str]])
    #    clean_and_tokenize = (소문자화 → 특수문자 제거 → 토큰화 → 불용어/1글자 제거)
    return [clean_and_tokenize(t) for t in text_list]


# 🔹 학습/검증/테스트 코퍼스를 토큰 시퀀스로 변환
train_tokens = tokenize_corpus(train_texts)  # List[str] → List[List[str]]
val_tokens = tokenize_corpus(val_texts)
test_tokens = tokenize_corpus(test_texts)

# 🔹 각 학습 문서의 토큰 길이를 수집 (시퀀스 길이 분포 파악)
lengths = [len(toks) for toks in train_tokens]

# 🔹 길이 상한 후보를 95퍼센타일로 설정
#    너무 긴 꼬리(long tail) 문서 때문에 패딩/메모리 낭비가 커지는 것을 방지
max_len = int(np.percentile(lengths, 95))

# 🔹 최종 MAX_LEN 규칙:
#    최소 50토큰은 보장, 최대 400토큰까지만 허용, 그 사이에서 95퍼센타일 값을 사용
#    → 과도한 패딩/잘림을 완화하고, 계산량과 정보 보존의 균형을 맞춤
MAX_LEN = max(50, min(max_len, 400))

# 🔹 선택된 시퀀스 길이 설정을 로그로 확인
print('95th percentile length:', max_len, '| Using MAX_LEN =', MAX_LEN)


## 4. Vocabulary & 시퀀스 변환
- 자주 등장하는 토큰으로 어휘 사전 구축
- `<pad>` / `<unk>` 토큰 예약
- 텍스트 -> 인덱스 시퀀스 -> 고정 길이 패딩

In [ ]:
def build_vocab(token_lists: List[List[str]], min_freq: int = 3, max_size: int = 30000) -> Dict[str, int]:
    # ✅ 코퍼스 전체 토큰 빈도 집계: 이중 루프 전개(comprehension)로 평탄화 후 Counter
    counter = Counter(tok for tokens in token_lists for tok in tokens)
    # ✅ 상위 max_size개만 추출(빈도 내림차순)
    most_common = counter.most_common(max_size)
    # ✅ 특별 토큰 사전 초기화: 패딩(<pad>=0), 미등록어(<unk>=1) 인덱스 고정
    vocab = {'<pad>': 0, '<unk>': 1}
    for word, freq in most_common:
        # ✅ 최소 등장 빈도 미만(min_freq) 단어는 제외
        if freq < min_freq:
            continue
        # ✅ 이미 존재하지 않으면 현재 길이(=다음 인덱스)로 등록
        vocab.setdefault(word, len(vocab))
    return vocab


# ✅ 학습 코퍼스에서 단어 사전 생성: 매우 드문 단어 제외(min_freq=5), 상한 40,000
vocab = build_vocab(train_tokens, min_freq=5, max_size=40000)
print('Vocabulary size:', len(vocab))


def tokens_to_indices(tokens: List[str], vocab: Dict[str, int], max_len: int) -> List[int]:
    # ✅ 미등록어 인덱스 캐싱(빠른 접근)
    unk = vocab['<unk>']
    # ✅ 토큰→인덱스 매핑: vocab에 없으면 <unk> 인덱스 사용; 그리고 최대 길이로 잘라냄(truncation)
    idxs = [vocab.get(tok, unk) for tok in tokens][:max_len]
    # ✅ 패딩: max_len보다 짧으면 <pad>(0)로 오른쪽 패딩하여 고정 길이 시퀀스 구성
    if len(idxs) < max_len:
        idxs += [vocab['<pad>']] * (max_len - len(idxs))
    return idxs


# ✅ 학습/검증/테스트 토큰 시퀀스를 모두 정수 인덱스 시퀀스로 변환(길이 MAX_LEN 고정)
train_indices = [tokens_to_indices(t, vocab, MAX_LEN) for t in train_tokens]
val_indices   = [tokens_to_indices(t, vocab, MAX_LEN) for t in val_tokens]
test_indices  = [tokens_to_indices(t, vocab, MAX_LEN) for t in test_tokens]


## 5. 임베딩 생성 유틸리티
Word2Vec / FastText 학습, GloVe 로딩을 함수화합니다.

In [ ]:
EMBED_DIM = 100  # 🔧 임베딩 벡터 차원 수(Word2Vec/FastText/GloVe 공통 기본값). 모델·행렬 크기를 결정

def train_word2vec(sentences: List[List[str]], vector_size: int = EMBED_DIM) -> Word2Vec:
    # ✅ Word2Vec 임베딩 학습 함수
    # - sentences: 토큰화된 문장들의 리스트(코퍼스)
    # - vector_size: 단어 벡터 차원(기본 EMBED_DIM)
    # - window=5: 목표 단어 좌우로 5개의 문맥 단어 고려
    # - min_count=5: 등장 빈도 5 미만 단어는 무시(노이즈 감소)
    # - workers=4: 멀티코어 병렬 학습 스레드 수
    # - sg=1: Skip-gram(1) / CBOW(0). Skip-gram은 희귀 단어 학습에 상대적으로 강함
    model = Word2Vec(sentences=sentences, vector_size=vector_size, window=5, min_count=5, workers=4, sg=1)
    return model  # 학습된 Gensim Word2Vec 모델 반환

def train_fasttext(sentences: List[List[str]], vector_size: int = EMBED_DIM) -> FastText:
    # ✅ FastText 임베딩 학습 함수
    # - FastText는 서브워드(문자 n-gram) 단위로 임베딩을 학습 → 희귀/신조어 처리에 유리
    # - 파라미터 의미는 Word2Vec과 유사(Window, min_count, workers 등)
    model = FastText(sentences=sentences, vector_size=vector_size, window=5, min_count=5, workers=4)
    return model  # 학습된 Gensim FastText 모델 반환

def load_glove(vector_size: int = EMBED_DIM):
    # ✅ 사전학습(pretrained) GloVe 임베딩 로드
    # - torchtext가 환경에 설치/호환되어 있어야 함(TORCHTEXT_AVAILABLE 플래그)
    if not TORCHTEXT_AVAILABLE:
        # 사용 불가 시 명확한 예외 메시지로 수동 로드 유도
        raise RuntimeError('torchtext를 사용할 수 없습니다. GloVe 벡터 파일을 수동으로 로드하세요.')
    # name='6B': 6 Billion 토큰 코퍼스에서 학습된 GloVe 세트 사용
    # dim=vector_size: 임베딩 차원(50/100/200/300 등 중 제공)
    glove = GloVe(name='6B', dim=vector_size)
    return glove  # torchtext.vocab.Vectors와 유사한 객체(단어→인덱스, 벡터 텐서 포함)

def build_embedding_matrix(vocab: Dict[str, int], embeddings, kind: str) -> np.ndarray:
    # ✅ 어휘집(vocab)의 각 단어 인덱스에 맞춰 "임베딩 행렬"을 생성
    # 행렬 크기: (vocab 크기, EMBED_DIM). 기본은 작은 정규분포 난수로 초기화
    matrix = np.random.normal(scale=0.02, size=(len(vocab), EMBED_DIM))
    # <pad> 토큰(패딩)은 반드시 영벡터(0)로 고정 → 패딩이 모델에 신호를 주지 않게 함
    matrix[vocab['<pad>']] = np.zeros(EMBED_DIM)

    # 🔁 어휘집을 순회하며, 해당 단어의 사전학습/학습된 임베딩 벡터를 찾아 행렬에 매핑
    for word, idx in vocab.items():
        if word in ('<pad>', '<unk>'):
            # 패딩/미등록 토큰은 스킵(<pad>는 위에서 0으로 설정, <unk>는 초기 난수 유지)
            continue
        vector = None

        # 🔹 Gensim Word2Vec/FastText: embeddings.wv에 단어 존재 여부 확인 후 벡터 취득
        if kind == 'word2vec' and word in embeddings.wv:
            vector = embeddings.wv[word]
        elif kind == 'fasttext' and word in embeddings.wv:
            vector = embeddings.wv[word]

        # 🔹 torchtext GloVe: stoi(단어→인덱스)로 위치 조회 후 vectors[인덱스]에서 벡터 취득
        elif kind == 'glove':
            if hasattr(embeddings, 'vectors'):       # 안전성 체크(환경별 구현 차이)
                token_index = embeddings.stoi.get(word)
                if token_index is not None:
                    vector = embeddings.vectors[token_index].numpy()

        # 📥 벡터를 찾았으면 해당 단어 인덱스 위치에 복사(초기 난수값을 대체)
        if vector is not None:
            matrix[idx] = vector

    # ✅ float32로 캐스팅(메모리 효율, PyTorch 호환성)
    return matrix.astype(np.float32)


## 6. PyTorch Dataset / DataLoader
시퀀스와 레이블을 텐서로 감싸는 간단한 Dataset을 정의하고, DataLoader를 구성합니다.

In [ ]:
class NewsDataset(Dataset):
    # ✅ PyTorch의 기본 Dataset 클래스를 상속받아 "뉴스 분류용 커스텀 데이터셋" 정의
    # Dataset은 DataLoader가 데이터를 배치(batch) 단위로 꺼낼 수 있도록 인터페이스를 제공합니다.

    def __init__(self, sequences: List[List[int]], labels: List[int]):
        # 🔹 sequences: 각 문서가 정수 인덱스로 변환된 토큰 시퀀스 (shape: [num_samples, max_len])
        # 🔹 labels: 각 문서의 정답 라벨(정수형 카테고리 ID)
        # 🔹 torch.tensor(...) : 넘파이 리스트를 텐서로 변환해야 GPU 연산 및 DataLoader 호환 가능
        self.sequences = torch.tensor(sequences, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        # ✅ 전체 데이터의 샘플 개수를 반환
        # DataLoader가 몇 개 배치를 만들 수 있는지 계산할 때 사용됩니다.
        return len(self.labels)

    def __getitem__(self, idx):
        # ✅ 하나의 샘플을 인덱스로 불러오는 메서드
        # DataLoader는 배치를 구성할 때 내부적으로 __getitem__을 반복 호출합니다.
        # idx번째 문장의 토큰 시퀀스와 정답 라벨을 (X, y) 형태로 반환합니다.
        return self.sequences[idx], self.labels[idx]

# ✅ NewsDataset 객체 생성
# 정수 인덱스로 변환된 문장과 라벨을 각각 학습/검증/테스트 세트로 구성합니다.
train_dataset = NewsDataset(train_indices, train_labels)
val_dataset = NewsDataset(val_indices, val_labels)
test_dataset = NewsDataset(test_indices, test_labels)

# ✅ DataLoader: Dataset을 감싸서 배치(batch) 단위로 데이터를 제공하는 유틸리티
# - batch_size: 한 번에 모델에 공급할 샘플 수
# - shuffle=True: 학습 시 데이터 순서를 섞어서 과적합 방지
# - DataLoader는 __getitem__을 자동으로 반복 호출하여 배치 텐서를 구성합니다.

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # 학습용 데이터 (랜덤 셔플)
val_loader = DataLoader(val_dataset, batch_size=64)                    # 검증용 데이터 (순서 유지)
test_loader = DataLoader(test_dataset, batch_size=64)                  # 테스트용 데이터 (순서 유지)

## 7. RNN 분류 모델
- 임베딩 레이어(사전학습 가중치 로드 가능)
- 양방향 LSTM
- 풀링 + FC 분류기

In [ ]:
class TextClassifierRNN(nn.Module):
    # ✅ PyTorch 기반 텍스트 분류 모델 정의 (RNN-LSTM 구조)
    # - 입력: 단어 인덱스 시퀀스(batch_size × seq_len)
    # - 출력: 클래스별 로짓(logit) 값(batch_size × num_classes)

    def __init__(self, vocab_size: int, embed_dim: int, hidden_dim: int,
                 num_classes: int, pretrained_weights=None, freeze_embedding=False):
        # ------------------------------------------------------------
        # vocab_size: 전체 단어 사전 크기 (Embedding lookup table 행 개수)
        # embed_dim: 임베딩 벡터 차원 수
        # hidden_dim: LSTM의 은닉 상태(hidden state) 크기
        # num_classes: 예측할 클래스 개수 (출력 차원)
        # pretrained_weights: 사전학습된 임베딩 행렬 (Word2Vec/FastText/GloVe)
        # freeze_embedding: True면 임베딩을 학습시키지 않고 고정(freeze)
        # ------------------------------------------------------------
        super().__init__()  # nn.Module 초기화 (상위 클래스 생성자 호출)

        # 🔹 임베딩 레이어: 단어 인덱스를 실수 벡터로 변환
        # padding_idx=0 → '<pad>' 토큰은 학습되지 않고 항상 0벡터 유지
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # 🔹 사전학습된 임베딩 가중치가 있다면 해당 행렬을 복사
        if pretrained_weights is not None:
            # pretrained_weights는 numpy 배열 형태이므로 torch.tensor로 변환 후 복사
            self.embedding.weight.data.copy_(torch.tensor(pretrained_weights))

        # 🔹 freeze 설정: False → 학습 가능 / True → 가중치 고정
        # requires_grad=False면 역전파(backpropagation) 시 업데이트되지 않음
        self.embedding.weight.requires_grad = not freeze_embedding

        # 🔹 양방향 LSTM 인코더
        # - embed_dim: 입력 차원 (단어 벡터 크기)
        # - hidden_dim: 은닉 상태 크기
        # - batch_first=True → 입력 형태를 (batch, seq, feature)로 지정
        # - bidirectional=True → 양방향 LSTM (정방향 + 역방향) 사용
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)

        # 🔹 Dropout: 과적합 방지를 위해 일부 뉴런을 확률적으로 30% 끔
        self.dropout = nn.Dropout(0.3)

        # 🔹 분류기(Linear Layer): 은닉 상태의 출력을 클래스 수만큼 매핑
        # 양방향 LSTM의 출력은 hidden_dim * 2이므로 입력 차원을 2배로 지정
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        # ✅ 순전파(forward) 연산 정의
        # 입력 x: (batch_size, seq_len) 형태의 단어 인덱스 텐서

        # 1️⃣ 단어 인덱스 → 임베딩 벡터 변환
        #   결과: (batch_size, seq_len, embed_dim)
        embeds = self.embedding(x)

        # 2️⃣ LSTM 인코더 통과
        #   outputs: 각 시점의 은닉 상태 전체 시퀀스 (batch, seq_len, hidden_dim*2)
        #   _: (h_n, c_n) 은닉/셀 상태 (여기서는 사용하지 않음)
        outputs, _ = self.encoder(embeds)

        # 3️⃣ 시퀀스 차원 평균 풀링
        #   모든 타임스텝의 은닉 상태를 평균내어 문장 전체 표현을 생성
        #   (batch_size, hidden_dim*2)
        pooled = outputs.mean(dim=1)

        # 4️⃣ Dropout → Linear 분류기 통과 → 로짓(logit) 출력
        #   logits: (batch_size, num_classes)
        logits = self.classifier(self.dropout(pooled))

        # 5️⃣ Softmax 이전 단계의 로짓 반환
        #   (손실 계산 시 CrossEntropyLoss가 내부에서 softmax 처리)
        return logits



## 8. 학습 / 검증 루프 Helper

In [ ]:
def run_epoch(model, loader, criterion, optimizer=None):
    # ✅ 한 epoch(데이터 전체 1회 반복) 동안 모델을 학습 또는 평가하는 함수
    # model: 학습시킬 신경망 모델
    # loader: DataLoader (train/val/test 중 하나)
    # criterion: 손실 함수 (예: CrossEntropyLoss)
    # optimizer: 옵티마이저 (없으면 평가 모드로 작동)

    # 🔹 학습 모드 여부 결정 (optimizer가 있으면 학습, 없으면 평가)
    is_train = optimizer is not None

    # 🔹 모델 모드 설정: 학습(model.train()) / 평가(model.eval())
    # train(): Dropout, BatchNorm 활성 / eval(): 평가 시 고정
    model.train() if is_train else model.eval()

    # 🔹 전체 손실 누적값, 예측/정답 저장용 리스트 초기화
    total_loss = 0
    all_preds, all_labels = [], []

    # 🔹 DataLoader에서 배치 단위로 데이터 반복
    for inputs, targets in loader:
        # inputs: (batch_size, seq_len) 형태의 단어 인덱스 시퀀스
        # targets: (batch_size,) 형태의 정답 클래스 인덱스
        # GPU/CPU 장치로 이동
        inputs, targets = inputs.to(device), targets.to(device)

        # 🔹 학습 모드일 때만 gradient 계산 활성화, 평가 시 비활성화(no_grad와 동일 효과)
        with torch.set_grad_enabled(is_train):
            # ① 순전파(forward)
            logits = model(inputs)              # (batch_size, num_classes)
            loss = criterion(logits, targets)   # CrossEntropyLoss 계산

            # ② 학습 단계일 경우: 역전파 + 최적화
            if is_train:
                optimizer.zero_grad()                 # 이전 그래디언트 초기화
                loss.backward()                       # 손실에 대해 역전파 수행
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # 🔧 Gradient 폭주 방지 (max norm=1.0)
                optimizer.step()                       # 파라미터 업데이트

        # 🔹 손실 누적 (평균 대신 총합 * 배치 크기 → 나중에 전체 평균 계산)
        total_loss += loss.item() * inputs.size(0)

        # 🔹 예측값 계산 (가장 높은 logit의 인덱스를 선택)
        preds = torch.argmax(logits, dim=1)
        # 🔹 CPU로 이동 후 리스트에 저장 (나중에 전체 연결)
        all_preds.append(preds.cpu())
        all_labels.append(targets.cpu())

    # 🔹 평균 손실 계산 (총 손실 ÷ 전체 샘플 수)
    avg_loss = total_loss / len(loader.dataset)

    # 🔹 Accuracy 계산
    # torch.cat으로 배치별 결과를 하나로 합친 후 sklearn의 accuracy_score로 계산
    accuracy = accuracy_score(torch.cat(all_labels), torch.cat(all_preds))

    # 🔹 평균 손실과 정확도 반환
    return avg_loss, accuracy

def evaluate_model(model, loader):
    # ✅ 모델의 최종 성능 평가 함수
    # loader: 검증/테스트용 DataLoader
    # 출력: classification_report(precision, recall, f1-score 등)

    model.eval()  # 🔹 평가 모드 (Dropout 비활성화)
    all_preds, all_labels = [], []

    # 🔹 no_grad(): 평가 시 gradient 계산 비활성화 → 메모리 절약 & 속도 향상
    with torch.no_grad():
        for inputs, targets in loader:
            # 입력을 디바이스로 이동 (targets은 나중에 CPU에서 비교)
            inputs = inputs.to(device)
            # 순전파
            logits = model(inputs)
            # 각 샘플별 예측 클래스 (argmax)
            preds = torch.argmax(logits, dim=1)
            # 결과 저장
            all_preds.append(preds.cpu())
            all_labels.append(targets)

    # 🔹 전체 예측/정답 배열 연결
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()

    # 🔹 정밀도(Precision), 재현율(Recall), F1-score 등 상세 지표 출력
    print(classification_report(y_true, y_pred, target_names=label_names))


## 9. 임베딩별 실험 함수

In [ ]:
def run_experiment(embedding_type: str, epochs: int = 3, freeze_embedding: bool = False):
    # ✅ 하나의 임베딩 방식(Word2Vec/FastText/GloVe)을 선택해
    #    벡터 학습/로딩 → 임베딩 행렬 구성 → RNN 분류기 학습/검증/테스트까지
    #    "한 번의 실험" 전체 파이프라인을 실행하는 함수
    # embedding_type: 'word2vec' | 'fasttext' | 'glove'
    # epochs: 학습 에폭 수
    # freeze_embedding: 임베딩 고정 여부(True면 학습 중 임베딩 가중치 업데이트 안 함)

    if embedding_type == 'word2vec':
        # 🔹 학습 코퍼스로 Word2Vec 임베딩을 직접 학습
        vectors = train_word2vec(train_tokens)
    elif embedding_type == 'fasttext':
        # 🔹 학습 코퍼스로 FastText 임베딩을 직접 학습(서브워드 기반)
        vectors = train_fasttext(train_tokens)
    elif embedding_type == 'glove':
        # 🔹 사전학습된(pretrained) GloVe 벡터 로드 (torchtext 필요)
        vectors = load_glove()
    else:
        # 🔹 안전장치: 지원하지 않는 문자열 입력 시 즉시 실패
        raise ValueError('지원하지 않는 임베딩 타입입니다.')

    # 🔹 vocab(단어→인덱스)에 맞춘 임베딩 행렬 생성
    #    - shape: (vocab_size, EMBED_DIM)
    #    - <pad>는 0벡터, <unk>는 초기 난수 유지
    #    - vectors에서 해당 단어 벡터를 찾으면 복사, 없으면 초기값 유지
    embedding_matrix = build_embedding_matrix(vocab, vectors, embedding_type)

    # 🔹 텍스트 분류 모델(RNN: BiLSTM) 구성
    #    - vocab_size: 임베딩 테이블 크기
    #    - embed_dim: 임베딩 차원(EMBED_DIM과 일치)
    #    - hidden_dim: LSTM 은닉 상태 크기(양방향이라 최종 분류 입력은 hidden_dim*2)
    #    - num_classes: 레이블 수(20 Newsgroups = 20)
    #    - pretrained_weights: 위에서 만든 임베딩 행렬로 초기화
    #    - freeze_embedding: True면 임베딩 고정(미세조정 없음)
    model = TextClassifierRNN(
        vocab_size=len(vocab),
        embed_dim=EMBED_DIM,
        hidden_dim=128,
        num_classes=len(label_names),
        pretrained_weights=embedding_matrix,
        freeze_embedding=freeze_embedding
    ).to(device)  # 🔹 모델을 GPU/CPU 장치로 이동

    # 🔹 손실 함수: 다중분류용 CrossEntropyLoss(내부적으로 softmax 포함)
    criterion = nn.CrossEntropyLoss()
    # 🔹 옵티마이저: Adam (기본 lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    history = []  # 🔹 에폭별 성능 로그를 기록(시각화/분석용)

    for epoch in range(1, epochs + 1):
        # 🔁 학습 에폭 수행: train_loader에서 손실/정확도 계산 + 역전파/업데이트
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer)
        # 🔁 검증 에폭 수행: val_loader에서 손실/정확도 계산(가중치 업데이트 없음)
        val_loss, val_acc = run_epoch(model, val_loader, criterion)

        # 🔹 로깅: 나중에 최고 성능, 학습 곡선 시각화 등에 사용
        history.append({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc,
        })

        # 🔹 진행 상황 출력(임베딩 타입/에폭/정확도)
        print(f"[{embedding_type}] Epoch {epoch}: train_acc={train_acc:.4f} val_acc={val_acc:.4f}")

    # 🔹 검증 정확도의 최고치 요약 출력
    print(f"[{embedding_type}] 검증 성능 최고치: {max(h['val_acc'] for h in history):.4f}")

    # 🔹 최종 테스트 리포트 출력(precision/recall/f1 등 클래스별 지표)
    print('테스트 리포트:')
    evaluate_model(model, test_loader)

    # 🔹 학습 곡선(손실/정확도) 등 사후 분석을 위해 히스토리 반환
    return history


## 10. 실행 가이드
아래 셀에서 원하는 임베딩 유형을 선택해 순차적으로 실험할 수 있습니다. (학습 시간 문제로 하나씩 실행 권장)
```python
word2vec_history = run_experiment('word2vec', epochs=5)
fasttext_history = run_experiment('fasttext', epochs=5)
# GloVe는 torchtext가 벡터를 다운로드할 수 있을 때만 사용하세요.
# glove_history = run_experiment('glove', epochs=5, freeze_embedding=True)
```


## 11. 개별 실험 실행 셀
각 임베딩을 **순차적으로** 실행해 결과를 비교합니다. 먼저 공용 이력을 저장할 딕셔너리를 초기화한 뒤, Word2Vec → FastText → GloVe 순으로 셀을 실행하세요.

In [ ]:
experiment_histories = {}
print('실험 이력 딕셔너리를 초기화했습니다.')

### 11-1. Word2Vec + LSTM
가장 먼저 Word2Vec 임베딩을 학습해 LSTM 분류기를 학습합니다.

In [ ]:
print('=== Word2Vec 실험 시작 ===')
word2vec_history = run_experiment('word2vec', epochs=5)
experiment_histories['word2vec'] = word2vec_history
print('Word2Vec 실험 완료')

### 11-2. FastText + LSTM (필요 시 GRU로 확장 가능)
FastText 임베딩을 동일 구조에 적용해 결과를 비교합니다.

In [ ]:
print('=== FastText 실험 시작 ===')
fasttext_history = run_experiment('fasttext', epochs=5)
experiment_histories['fasttext'] = fasttext_history
print('FastText 실험 완료')

### 11-3. GloVe + LSTM
TorchText가 제공하는 사전학습 GloVe 벡터를 사용할 수 있을 때 실행하세요.

In [ ]:
if TORCHTEXT_AVAILABLE:
    print('=== GloVe 실험 시작 ===')
    glove_history = run_experiment('glove', epochs=5, freeze_embedding=True)
    experiment_histories['glove'] = glove_history
    print('GloVe 실험 완료')
else:
    print('torchtext가 설치/다운로드되지 않아 GloVe 실험을 건너뜁니다.')

## 12. 결과 요약 테이블
여러 임베딩의 `val_acc` 최고값을 모아 비교합니다.

In [ ]:
import pandas as pd

rows = []
for name, history in experiment_histories.items():
    best = max(history, key=lambda h: h['val_acc'])
    rows.append({
        'embedding': name,
        'best_epoch': best['epoch'],
        'best_val_acc': best['val_acc'],
        'train_acc_at_best': best['train_acc']
    })

results_df = pd.DataFrame(rows)
display(results_df.sort_values('best_val_acc', ascending=False))


## 13. 추가 분석 & 메모
- `classification_report` 출력 로그를 바탕으로 클래스별 정밀도/재현율을 비교합니다.
- 더 높은 성능을 위해서는 데이터 정제 강화, 하이퍼파라미터 조정, GRU/어텐션 추가 등을 실험해 보세요.
- 아래 마크다운 셀에 성능 비교 및 인사이트를 정리해 제출 파일에 포함하세요.

**결과 정리 예시**
- Word2Vec: Best Val Acc = ... (epoch ...). 주요 오차 패턴은 ...
- FastText: ...
- GloVe: ...

향후 개선 아이디어: ...